In [1]:
from appgeopy import *
from my_packages import *
from run_workflow import *
from sklearn.preprocessing import StandardScaler

#### 1

we have some gaps in the original data, so we need to fill them with PCA

#### 2

In [2]:
mlcw_fpath = r"20251114_MLCW_CRFP_monthly_v4.h5"
mlcw_obj = MLCW(mlcw_fpath)

mlcw_measures, mlcw_metadata = mlcw_obj.get_data()

available_stations = mlcw_obj.list_stations()
available_stations[:5]

['ANHE', 'ANNAN', 'BEICHEN', 'CANLIN', 'DONGGUANG']

In [3]:
combined_df = pd.DataFrame(data=None, index=None, dtype=np.float32)

# select_station = available_stations[0]

number_of_layers = []
for select_station in tqdm(available_stations[:]):

    string_decoder = lambda arr: [x.decode("utf-8") for x in arr]

    measures_byStation = mlcw_measures[select_station]
    monthly_date_arr = pd.to_datetime(
        string_decoder(measures_byStation["monthly_date"])
    )
    monthly_values_arr = measures_byStation["monthly_values"][
        "compactbylayer_PCA"
    ]

    cdisp_mlcw_df = pd.DataFrame(data={"time": monthly_date_arr})

    n_layers = monthly_values_arr.shape[0]
    number_of_layers.append(n_layers)

    for i in range(n_layers):
        cdisp_mlcw_df[f"Layer_{i+1}"] = monthly_values_arr[i]

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # 2025/4/15 : decide to keep working with cumulative displacement
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    cdisp_mlcw_df = cdisp_mlcw_df.set_index("time")
    # cdisp_mlcw_df = cdisp_mlcw_df.interpolate(method="time")

    cdisp_mlcw_df = cdisp_mlcw_df.loc[:, :]

    first_datetime = cdisp_mlcw_df.first_valid_index()
    last_datetime = cdisp_mlcw_df.last_valid_index()

    cdisp_mlcw_df = cdisp_mlcw_df.loc[first_datetime:last_datetime, :]

    y_twd97 = mlcw_metadata[select_station]["X_TWD97"]
    x_twd97 = mlcw_metadata[select_station]["Y_TWD97"]

    cdisp_mlcw_df.insert(
        loc=0, column="Y_TWD97", value=[y_twd97] * len(cdisp_mlcw_df)
    )
    cdisp_mlcw_df.insert(
        loc=0, column="X_TWD97", value=[x_twd97] * len(cdisp_mlcw_df)
    )
    cdisp_mlcw_df.insert(
        loc=0, column="STATION", value=[select_station] * len(cdisp_mlcw_df)
    )

    combined_df = pd.concat([combined_df, cdisp_mlcw_df], axis=0)
    # - - - - - - - - - - - - - - - - - - - - - - - - -
    # 2025/4/8 : convert to displacement time series
    # - - - - - - - - - - - - - - - - - - - - - - - - -
    # disp_mlcw_df = cdisp_mlcw_df.diff(axis=0)
    # disp_mlcw_df = disp_mlcw_df.loc["2014":, :]

    # first_datetime = disp_mlcw_df.first_valid_index()
    # last_datetime = disp_mlcw_df.last_valid_index()

    # disp_mlcw_df = disp_mlcw_df.loc[first_datetime:last_datetime, :]
    # disp_mlcw_df.insert(loc=0, column="STATION", value=[select_station]*len(disp_mlcw_df))

    # combined_df = pd.concat([combined_df, disp_mlcw_df], axis=0)

combined_df.to_pickle(r"Monthly_MLCW_pca_CUMDISP_v3.xz")

  0%|          | 0/39 [00:00<?, ?it/s]

In [4]:
combined_df

,STATION,X_TWD97,Y_TWD97,Layer_1,Layer_2,Layer_3,Layer_4
time,,,,,,,
2004-11-01,ANHE,2.602035e+06,179539.204623,0.000000,0.000000,0.000000,0.000000
2004-12-01,ANHE,2.602035e+06,179539.204623,-3.000000,4.000000,-1.000000,4.000000
2005-01-01,ANHE,2.602035e+06,179539.204623,-0.293140,-2.031953,3.554529,1.041336
2005-02-01,ANHE,2.602035e+06,179539.204623,-1.286995,-3.254393,1.902571,0.811722
2005-03-01,ANHE,2.602035e+06,179539.204623,-2.537195,-4.753250,-0.359285,0.440431
...,...,...,...,...,...,...,...
2021-07-01,ZHUTANG,2.639635e+06,191944.074820,-28.000000,-116.000000,-88.000000,-41.000000
2021-08-01,ZHUTANG,2.639635e+06,191944.074820,-29.000000,-117.000000,-89.000000,-41.000000
2021-09-01,ZHUTANG,2.639635e+06,191944.074820,-29.000000,-117.000000,-89.000000,-41.000000
